In [1]:
%matplotlib inline
%config Completer.use_jedi=False

import os; os.environ['KERAS_BACKEND'] = 'theano'
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils import shuffle
from scipy import ndimage
from skimage.transform import resize


# Local imports
import adjust_path  # Before doing any local imports
from icc.data_loader import DataLoader
from icc.models.gangsta_net import GangstaNet

Appending "/code/notebooks/.." to path


In [2]:
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.


In [124]:
X, y = DataLoader.load_train()
X['inc_angle_missing'] = X.inc_angle.map(pd.isnull)
X.inc_angle.fillna(value=0, inplace=True)
xTrain, xTest, yTrain, yTest = train_test_split(X.copy(), y.copy(), test_size=0.15)

In [125]:
def transform(df):
    images = []
    for i, row in df.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 / band_2
    
        band_1_norm = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        band_2_norm = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        band_3_norm = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())
        img = np.dstack((band_1_norm, band_2_norm, band_3_norm)) ** 2
        images.append(img)
    return np.array(images)

def augment(images):
    image_mirror_lr = []
    image_mirror_ud = []
    for i in range(0,images.shape[0]):
        band_1 = images[i,:,:,0]
        band_2 = images[i,:,:,1]
        band_3 = images[i,:,:,2]
            
        # mirror left-right
        band_1_mirror_lr = np.flip(band_1, 0)
        band_2_mirror_lr = np.flip(band_2, 0)
        band_3_mirror_lr = np.flip(band_3, 0)
        image_mirror_lr.append(np.dstack((band_1_mirror_lr, band_2_mirror_lr, band_3_mirror_lr)))
        
        # mirror up-down
        band_1_mirror_ud = np.flip(band_1, 1)
        band_2_mirror_ud = np.flip(band_2, 1)
        band_3_mirror_ud = np.flip(band_3, 1)
        image_mirror_ud.append(np.dstack((band_1_mirror_ud, band_2_mirror_ud, band_3_mirror_ud)))
        
    mirrorlr = np.array(image_mirror_lr)
    mirrorud = np.array(image_mirror_ud)
    images = np.concatenate((images, mirrorlr, mirrorud))
    return images

xTrainImgs = transform(xTrain[['band_1', 'band_2']])
xTestImgs = transform(xTest[['band_1', 'band_2']])

xTrainImgs = augment(xTrainImgs)
yTrain = np.concatenate((yTrain, yTrain, yTrain))
print(xTrainImgs.shape)
print(yTrain.shape)

(4089, 75, 75, 3)
(4089,)


In [126]:
class DoubleNet(nn.Module):
    
    def __init__(self):
        super(DoubleNet, self).__init__()
        
        torch.manual_seed(0)
        
        self.conv_out_shape = 2592
        
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Input 1 branch
        self.input1_layer1_conv2d = nn.Conv2d(in_channels=3, out_channels=32,  kernel_size=5, padding=3)
        self.input1_layer2_conv2d = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.input1_layer3_conv2d = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        
        self.input1_layer4_fc = nn.Linear(in_features=self.conv_out_shape, out_features=512)
        
        # Concatenated branch, fully connected stream
        self.fc1 = nn.Linear(in_features=512, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=1)
        
    def forward(self, input1, training=False):
        
        input1 = self.max_pool(F.relu(self.input1_layer1_conv2d(input1)))
        input1 = F.dropout2d(input1, p=0.2, training=training)
        
        input1 = self.max_pool(F.relu(self.input1_layer2_conv2d(input1)))
        input1 = F.dropout2d(input1, p=0.2, training=training)
        
        input1 = self.max_pool(F.relu(self.input1_layer3_conv2d(input1)))
        input1 = F.dropout2d(input1, p=0.2, training=training)
        
        input1 = F.relu(self.input1_layer4_fc(input1.view(-1, self.conv_out_shape)))
        input1 = F.dropout2d(input1, p=0.2, training=training)
        
        combined = (self.fc1(input1))
        combined = F.dropout2d(combined, p=0.2, training=training)
        combined = self.fc2(combined)
        
        return combined

DoubleNet()

DoubleNet (
  (avg_pool): AvgPool2d (size=2, stride=2, padding=0, ceil_mode=False, count_include_pad=True)
  (max_pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (input1_layer1_conv2d): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(3, 3))
  (input1_layer2_conv2d): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (input1_layer3_conv2d): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (input1_layer4_fc): Linear (2592 -> 512)
  (fc1): Linear (512 -> 256)
  (fc2): Linear (256 -> 1)
)

In [127]:
# print('transfering to cuda')
print('initializing model')
net = DoubleNet()
net.cuda()

n_epoch = 35
batch_size = 100
img_size = 75

print('defining optimizers')
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.0)
criterion = nn.BCEWithLogitsLoss()

datagen_params = dict(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-6,
    rotation_range=0,
    width_shift_range=0.,
    height_shift_range=0.,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='wrap',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None,
    data_format='channels_last'
)

datagen = ImageDataGenerator(**datagen_params)
datagen.fit(xTrainImgs, augment=True, seed=123)

# Test data setup
testImgs = Variable(torch.FloatTensor(xTestImgs.reshape(-1, 3, 75, 75)).cuda())
testTarget = Variable(torch.FloatTensor(yTest.values.astype(float).reshape(-1, 1)).cuda(), requires_grad=False)


print('starting training')
for epoch in range(n_epoch):
    i = 0
    for img_batch, target in datagen.flow(xTrainImgs, yTrain):
        
        trainImgs = Variable(torch.FloatTensor(img_batch.reshape(-1, 3, 75, 75)).cuda())
        target = Variable(torch.FloatTensor(target.astype(float).reshape(-1, 1)).cuda())

        # batch step
        optimizer.zero_grad()

        out = net(trainImgs, training=True)
        loss = criterion(out, target)
        loss.backward()

        optimizer.step()
        
        i += 1
        if i % 5 == 0: 
            
            out = net(testImgs)
            train_loss = criterion(out, testTarget).data.cpu().numpy()[0]
            pct_complete = (i / 100) * 100
            sys.stdout.write('\rEpoch: {}, Train loss: {:.4f} - Test loss: {:.4f} - {:.1f}%'
                             .format(epoch+1, loss.data.cpu().numpy()[0], train_loss, pct_complete))
        if i > 100:
            sys.stdout.write('\rEpoch: {}, Train loss: {:.4f} - Test loss: {:.4f} - {:.1f}%'
                             .format(epoch+1, loss.data.cpu().numpy()[0], train_loss, 100))
            break
    
    print()
    

initializing model
defining optimizers
starting training
Epoch: 1, Train loss: 0.6268 - Test loss: 0.6160 - 100.0%
Epoch: 2, Train loss: 0.6698 - Test loss: 0.5742 - 100.0%
Epoch: 3, Train loss: 0.5632 - Test loss: 0.5754 - 100.0%
Epoch: 4, Train loss: 0.5877 - Test loss: 0.5684 - 100.0%
Epoch: 5, Train loss: 0.4747 - Test loss: 0.5513 - 100.0%
Epoch: 6, Train loss: 0.5337 - Test loss: 0.5496 - 100.0%
Epoch: 7, Train loss: 0.4857 - Test loss: 0.4773 - 100.0%
Epoch: 8, Train loss: 0.5768 - Test loss: 0.4986 - 100.0%
Epoch: 9, Train loss: 0.3977 - Test loss: 0.3635 - 100.0%
Epoch: 10, Train loss: 0.5031 - Test loss: 0.3659 - 100.0%
Epoch: 11, Train loss: 0.3246 - Test loss: 0.3672 - 100.0%
Epoch: 12, Train loss: 0.4706 - Test loss: 0.3715 - 100.0%
Epoch: 13, Train loss: 0.3843 - Test loss: 0.3384 - 100.0%
Epoch: 14, Train loss: 0.2797 - Test loss: 0.3171 - 100.0%
Epoch: 15, Train loss: 0.3131 - Test loss: 0.3204 - 100.0%
Epoch: 16, Train loss: 0.2761 - Test loss: 0.3284 - 100.0%
Epoch: 1

In [78]:
# network layer trials
img_size = 75

input1_layer1_conv2d = nn.Conv2d(in_channels=3, out_channels=32,  kernel_size=5, padding=3)
input1_layer2_conv2d = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
input1_layer3_conv2d = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)

fc1 = nn.Linear(in_features=2592, out_features=512)
pool_big = nn.MaxPool2d(kernel_size=3, stride=2, )
pool = nn.MaxPool2d(kernel_size=2, stride=2, )

pics = np.concatenate(xTrainImgs[:2], axis=0).reshape(-1, 3, img_size, img_size)
pics = torch.FloatTensor(pics)
pics = Variable(pics)

pics = pool(input1_layer1_conv2d(pics))
pics = pool(input1_layer2_conv2d(pics))
pics = pool(input1_layer3_conv2d(pics))

print(pics.size())
print(np.prod(pics.size()[1:]))

fc1(pics.view(-1, 2592))

torch.Size([2, 32, 9, 9])
2592


Variable containing:
-8.3421e-03 -1.1375e-02 -5.7629e-02  ...  -3.6862e-02  1.4998e-02  4.7157e-02
-1.8848e-02 -7.1759e-03 -6.2996e-02  ...  -4.3536e-02  8.2824e-03  3.9818e-02
[torch.FloatTensor of size 2x512]